In [7]:
import torch
import torch.nn as nn
from torch.func import vmap, grad, functional_call  # Nouvelle API PyTorch 2.x
import vutils

# Modèle simple
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear = nn.Linear(50, 10)
        self.linear2 = nn.Linear(10, 1)

    def forward(self, x):
        return self.linear2(self.linear(x))

# Initialisation
model = SimpleModel()
criterion = nn.MSELoss()

# Génération de données
batch_size = 5
x = torch.randn(batch_size, 50)  # Pas besoin de requires_grad=True ici
y = torch.randn(batch_size, 1)

gradient_mean, gradient_variance = vutils.get_grad_mean_var(model, criterion, x, y)

print(gradient_mean.keys())
# Affichage
p = 0.8
for name, var in gradient_variance.items():
    mean = gradient_mean[name]
    print(f"Seuils gradient pour {name}: {torch.sum(torch.distributions.Normal(0, 1).cdf(torch.abs(mean) / var) > p)}")

dict_keys(['linear.weight', 'linear.bias', 'linear2.weight', 'linear2.bias'])
Seuils gradient pour linear.weight: 362
Seuils gradient pour linear.bias: 10
Seuils gradient pour linear2.weight: 4
Seuils gradient pour linear2.bias: 0


In [9]:
list(model.parameters())


[Parameter containing:
 tensor([[-1.3782e-01, -6.0526e-02, -4.8069e-02, -8.9524e-03, -5.4907e-02,
          -3.6239e-02,  4.3580e-02,  1.3878e-01, -8.8553e-02, -3.5872e-02,
           1.3221e-01, -5.6092e-02,  1.9604e-02, -2.4736e-02,  1.3272e-01,
          -1.1607e-01, -6.8721e-02,  4.4338e-02,  8.3945e-02, -7.4793e-02,
           3.4903e-03,  4.3187e-02, -1.2079e-01, -7.7440e-02,  4.4152e-02,
           2.3509e-02, -1.0554e-01, -2.2795e-02, -2.3290e-02,  3.6980e-02,
           1.5180e-02, -5.4494e-02, -3.4270e-02,  1.2983e-01, -5.0781e-02,
           4.6702e-02,  7.4224e-02,  7.4471e-02,  1.3889e-01, -6.9914e-02,
           9.3390e-02, -1.5967e-02,  1.1818e-01,  4.0784e-03,  7.5586e-02,
          -1.0124e-01, -7.0380e-02,  1.0815e-01,  1.3227e-01,  1.0825e-01],
         [-2.6620e-03, -1.3538e-01,  3.2169e-02,  1.1253e-01, -1.1094e-01,
          -1.0437e-01,  7.8708e-02,  6.4943e-02, -7.0617e-02,  5.3161e-02,
           5.8821e-02,  4.6184e-02, -9.1995e-02, -7.2641e-02,  6.8709e-02,
 